In [4]:
import re
import cleantext
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
fnc_sample_raw = pd.read_csv('FakeNewsCorpus_Sample.csv', usecols=[*range(1,15)])

fnc_sample_raw['content'] = fnc_sample_raw['content'].str.lower()

print(fnc_sample_raw['content'])

AttributeError: 'Series' object has no attribute 'lower'